<a href="https://colab.research.google.com/github/gptix/DS-Unit-2-Applied-Modeling/blob/master/module2/Jud_Taylor_LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Read New York City property sales data
import pandas as pd
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Your code starts here:

In [0]:
# # Make backup of df
df_bak = df

# Break in case of trubble.
# df = df_bak

In [0]:
# Change column names: replace spaces with underscores
df.columns = [cn.replace(' ','_') for cn in df.columns]
# df.columns

In [0]:
# Get Pandas Profiling Report
import pandas_profiling
pfr = pandas_profiling.ProfileReport(df)
pfr.to_file("df_report.html") 

In [0]:
# Keep just the subset of data for the Tribeca neighborhood
# Check how many rows you have now. (Should go down from > 20k rows to 146)
# foo = df.NEIGHBORHOOD.unique().tolist()
# foo.sort()
# foo
df_tribeca = df[df['NEIGHBORHOOD'] == 'TRIBECA']
df_tribeca

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
220,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,"$ 2,800,000",2019-01-03 00:01:00
763,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,"$ 2,650,000",2019-01-07 00:01:00
996,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,212,1404,NaN,R4,"11 BEACH STREET, 1D",1D,10013.0,1.0,0.0,1.0,"11,213",2651.0,1900.0,2,R4,$ - 0,2019-01-08 00:01:00
1276,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,"$ 1,005,000",2019-01-09 00:01:00
1542,1,TRIBECA,02 TWO FAMILY DWELLINGS,1,132,26,NaN,S2,"75 WARREN STREET, XX",NaN,10007.0,2.0,1.0,3.0,"1,819",10117.0,1905.0,1,S2,"$ 12,950,000",2019-01-10 00:01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22221,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1206,NaN,R4,"443 GREENWICH STREET, 1F",1F,10013.0,1.0,0.0,1.0,"35,127",2429.0,1905.0,2,R4,"$ 5,761,259",2019-01-24 00:04:00
22732,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1806,NaN,R4,"70 LITTLE WEST STREET, 11K",11K,10004.0,1.0,0.0,1.0,0,1601.0,2006.0,2,R4,"$ 2,600,000",2019-01-29 00:04:00
22733,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,8071,NaN,R4,"2 SOUTH END AVENUE, 5G",5G,10280.0,1.0,0.0,1.0,0,634.0,1990.0,2,R4,"$ 605,000",2019-01-29 00:04:00
22897,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,8132,NaN,R4,"2 SOUTH END AVENUE, 8B",8B,10280.0,1.0,0.0,1.0,0,939.0,1990.0,2,R4,"$ 960,000",2019-01-30 00:04:00


In [0]:
# Q. What's the date range of these property sales in Tribeca?
 pd.to_datetime(df['SALE_DATE'],infer_datetime_format=True)

 import datetime as dt
 cn = 'SALE_DATE'
 fmt = '%M/%d/%Y'

# df['SALE_DATE'] = df['SALE_DATE'].apply(lambda x: dt.datetime.strptime(x,fmt))
df_tribeca[cn]

earliest = df_tribeca['SALE_DATE'].min().strftime('%Y-%m-%d')
earliest
latest = df_tribeca['SALE_DATE'].max().strftime('%Y-%m-%d')
latest

print(f'Date range of property sales in Tribeca: From {earliest} to {latest}.')
df_tribeca.shape

Date range of property sales in Tribeca: From 2019-01-01 to 2019-01-31.


(146, 21)

In [0]:
# The Pandas Profiling Report showed that SALE_PRICE was read as strings
# Convert it to integers

df_tribeca['SALE_PRICE'] = df_tribeca['SALE_PRICE'].replace('[\$\,\.\ ]', '',
                                                            regex=True).astype(int)
df_tribeca.SALE_PRICE

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


220       2800000
763       2650000
996             0
1276      1005000
1542     12950000
           ...   
22221     5761259
22732     2600000
22733      605000
22897      960000
22898      975000
Name: SALE_PRICE, Length: 146, dtype: int64

In [0]:
# Q. What is the maximum SALE_PRICE in this dataset?
max_sale_price = df_tribeca.SALE_PRICE.max()
print(max_sale_price)

260000000


In [0]:
# Look at the row with the max SALE_PRICE
df_tribeca[df_tribeca['SALE_PRICE'] == max_sale_price]

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
6499,1,TRIBECA,08 RENTALS - ELEVATOR APARTMENTS,2,224,1,NaN,D8,34 DESBROSSES STREET,NaN,10013.0,283.0,3.0,286.0,"36,858",305542.0,2007.0,2,D8,260000000,2019-01-01 00:02:00


In [0]:
# Get value counts of TOTAL_UNITS
# Q. How many property sales were for multiple units?
# df_tribeca[df_tribeca.TOTAL_UNITS > 1]
df_tribeca.TOTAL_UNITS.value_counts()

1.0      131
0.0       11
5.0        1
286.0      1
8.0        1
3.0        1
Name: TOTAL_UNITS, dtype: int64

In [0]:
# Keep only the single units
df_singles = df_tribeca[df_tribeca.TOTAL_UNITS == 1]
df_singles.shape
df_singles.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [0]:
# Q. Now what is the max sales price? How many square feet does it have?
print(f'${df_singles.SALE_PRICE.max():,}')
sqft = df_singles[df_singles.SALE_PRICE == df_singles.SALE_PRICE.max()].GROSS_SQUARE_FEET.iloc[0]
print(f'Square Feet: {sqft}')

$39,285,000
Square Feet: 8346.0


In [0]:
# Q. How often did $0 sales occur in this subset of the data?

# There's a glossary here: 
# https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

# It says:
# A $0 sale indicates that there was a transfer of ownership without a 
# cash consideration. There can be a number of reasons for a $0 sale including 
# transfers of ownership from parents to children. 

df_singles[df_singles.SALE_PRICE <=0].count().max()

15

In [0]:
# Look at property sales for > 5,000 square feet
# Q. What is the highest square footage you see?

df_singles[df_singles.GROSS_SQUARE_FEET > 5000].GROSS_SQUARE_FEET.max()

39567.0

In [111]:
# What are the building class categories?
# How frequently does each occur?

print(df_singles.BUILDING_CLASS_CATEGORY.value_counts())

13 CONDOS - ELEVATOR APARTMENTS               121
15 CONDOS - 2-10 UNIT RESIDENTIAL               8
16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT      1
46 CONDO STORE BUILDINGS                        1
Name: BUILDING_CLASS_CATEGORY, dtype: int64


In [124]:
# Keep subset of rows:
# Sale price more than $0, 
# Building class category = Condos - Elevator Apartments

# Check how many rows you have now. (Should be 106 rows.)
df_condos = df_singles[(df_singles['SALE_PRICE']>0) & (df_singles['BUILDING_CLASS_CATEGORY'] == '13 CONDOS - ELEVATOR APARTMENTS')]
df_condos.shape
# df_singles.shape
# df_singles['BUILDING_CLASS_CATEGORY']

(106, 21)

In [0]:
# Make a Plotly Express scatter plot of GROSS_SQUARE_FEET vs SALE_PRICE


In [0]:
# Add an OLS (Ordinary Least Squares) trendline,
# to see how the outliers influence the "line of best fit"


In [0]:
# Look at sales for more than $35 million

# All are at 70 Vestry Street
# All but one have the same SALE_PRICE & SALE_DATE
# Was the SALE_PRICE for each? Or in total?
# Is this dirty data?


In [0]:
# Make a judgment call:
# Keep rows where sale price was < $35 million

# Check how many rows you have now. (Should be down to 90 rows.)


In [0]:
# Now that you've removed outliers,
# Look again at a scatter plot with OLS (Ordinary Least Squares) trendline


In [0]:
# Select these columns, then write to a csv file named tribeca.csv. Don't include the index.
